In [21]:
import os
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

print("imported")

AI_MODEL="openai/gpt-4o-mini" # OR: openai/gpt-4o-mini /anthropic/claude-haiku-4-5-20251001/ change between them to observe the effects!
MAX_TOKENS_DEFAULT = 500

def get_completion(prompt_or_messages, model=AI_MODEL, max_tokens=MAX_TOKENS_DEFAULT, **kwargs):
    # If user passes a plain string, wrap it in a messages list
    if isinstance(prompt_or_messages, str):
        messages = [{"role": "user", "content": prompt_or_messages}]
    else:
        messages = prompt_or_messages

    # Guard: avoid sending empty messages
    if not isinstance(messages, list) or len(messages) == 0:
        raise ValueError(f"'messages' must be a non-empty list. Got: {messages}")

    resp = litellm.completion(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        **kwargs
    )
    return resp.choices[0].message.content

print("Functions are ready!")

# --- Our context here ---

bug_ticket_json = {
    "ticket_id": "108",
    "status": "Open",
    "prio": "Critical",
    "component": "Login API v2",
    "title": "Using default admin credentials, anyone can login and request any info from API",
    "logs": [
        {"timestamp": "2026-02-18T14:00:00z","level": "Error","message": "admin user credentials are exposed to public"},
        {"timestamp": "2026-02-18T14:02:00z","level": "Error","message": "admin user credentials are exposed to public"}
    ],
    "reporter": "george ghira",
}
# ---Convert the Python dict to a JSON string to be used in the prompt
bug_ticket_str = json.dumps(bug_ticket_json, indent=2)

print("All set!")


imported
Functions are ready!
All set!


In [22]:
unstructured_prompt = [
    {
         "role": "user", 
         "content": f'''
Please summarize the following bug ticket: {bug_ticket_str}, 
and it should start with the ticket ID and title. Mention prio and the effected component, 
but do not include the raw logs in the final summary.
'''
    }
]

response_unstructured = get_completion(unstructured_prompt)

In [23]:
print(response_unstructured)

**Ticket ID:** 108  
**Title:** Using default admin credentials, anyone can login and request any info from API  
**Priority:** Critical  
**Affected Component:** Login API v2  

A critical security issue has been reported regarding the Login API v2, where default admin credentials are publicly exposed, allowing unauthorized users to log in and access any information from the API. The ticket remains open and was reported by George Ghira.


In [24]:
prompt_with_markdown = dedent(f"""
#Instruction
Please summarize the following bug ticket.

#Ticket Data
{bug_ticket_str}

#Constrains
-The summary must be a single parahraph
-Start summary with ticket ID and title.
-Do not include the raw logs in the final summary.
""")

response_md = get_completion(prompt_with_markdown)

In [25]:
print(response_md)

Ticket ID 108, titled "Using default admin credentials, anyone can login and request any info from API," reports a critical issue with the Login API v2 where administrator credentials are publicly exposed, allowing unauthorized access to the API. The ticket is currently open and was reported by George Ghira.


In [27]:
prompt_with_xml = dedent(f"""
<Instruction>
Please summarize the following bug ticket.
</Instruction>

<Ticket Data>
{bug_ticket_str}
</Ticket Data>

<Constrains>
-The summary must be a single parahraph
-Start summary with ticket ID and title.
-Do not include the raw logs in the final summary.
</Constrains>
""")

response_xml = get_completion(prompt_with_xml)

In [28]:
print(response_xml)

Ticket ID 108, titled "Using default admin credentials, anyone can login and request any info from API," is currently open with a critical priority. The issue arises from exposed admin user credentials, allowing unauthorized access to the API, which poses significant security risks. The ticket was reported by George Ghira.


In [29]:
prompt_with_backticks = dedent(f"""

Please summarize the following bug ticket.


'''
{bug_ticket_str}
'''

Please follow this rules:
-The summary must be a single parahraph
-Start summary with ticket ID and title.
-Do not include the raw logs in the final summary.
""")

response_backticks = get_completion(prompt_with_backticks)

In [30]:
print(response_backticks)

Ticket ID 108, titled "Using default admin credentials, anyone can login and request any info from API," has been reported as a critical issue with the Login API v2. The problem arises from the exposure of admin user credentials to the public, allowing unauthorized access to sensitive information. The ticket is currently open and was reported by George Ghira.
